In [1]:
import requests

KBQA_URL = 'http://kbqa-service/HEALS/api/v1.0'
ENTITY_RESOLVER_URL = "http://fuzzy-entity-resolver/resolve"

In [2]:
import json
#
# Load user data from a json file
#
with open('1.json') as persona_file:
    persona_json = persona_file.read()
    persona = json.loads(persona_json)

In [3]:
#
# Set the base query
#
query = "what is a breakfast without eggs"
#query = "What are soul recipes that do not contain egg?"

In [4]:
#
# Expand the base query using user data
#
expanded_query_res = requests.post(f'{KBQA_URL}/expand_query', json={'question': query, 'persona_num': 1, 'persona': persona_json})
expanded_query = expanded_query_res.text
expanded_query

'what is a breakfast without eggs, and have milk,  apple,  pasta or  pizza, and contain calories from carbohydrates with desired range 0.0% to 45.0%?'

In [6]:
#
# Call the entity resolver to identify keywords in the query
#
entity_resp = requests.get(ENTITY_RESOLVER_URL, params={'query': query}).json()
tags = entity_resp['tags']

# Filter out tags with low confidence
tags = [t for t in tags if t['confidence'] >= 0.9]
for t in tags:
    t['confidence'] = 1
tags = [{'confidence': 1, 'entity': 'tag', 'label': 'breakfast', 'value': 'tag2305'}]
tags 

[{'confidence': 1, 'entity': 'tag', 'label': 'breakfast', 'value': 'tag2305'}]

In [7]:
constrained_entities = {
    1: [],
    2: [
        "egg",
        "eggs"
    ]
}
json.dumps(constrained_entities)

'{"1": [], "2": ["egg", "eggs"]}'

In [12]:
similar_recipes = [
    "Southern Smothered Pork Chops",
    "Creamy Cheese Grits",
    "Blueberry Peach Cobbler",
    "Oxtail Soup",
    "Seafood Boil",
    "Bubba Gump\\'s Greenbow County Okra Gumbo",
    "Easy Smothered Pork Chops",
    "Smothered Pork Chops",
    "Southern Cabbage Salad With Sweet Onion and Peppers",
    "Cabbage Rolls",
    "Easy Peach Cobbler",
    "Grilled Cabbage",
    "Cheese Grits",
    "Boozy Braised Collard Greens",
    "Stewed Okra and Tomatoes",
    "Okra Supreme",
    "Ham Hock Stock",
    "Fresh Peach Cobbler",
    "Pickled Okra",
    "Fried Green Tomatoes",
    "Collard Greens (Vegan)",
    "Fried Okra",
    "Okra Gumbo",
    "Crock Pot - Cabbage Beef Soup",
    "Oven-Fried Okra",
    "Chicken Gizzard Taco Filling",
    "Perfect Southern Greens (Kale, Beet, Collard Greens, Mustard)",
    "Peach Cobbler",
    "Lump Crab Garlic Cheese Grits",
    "Pan-Fried Okra, Onion, and Tomatoes",
    "Cabbage Soup",
    "Easy Vegetarian Collard Greens",
    "Baked Garlic Cheese Grits",
    "Cajun Smothered Pork Chops",
    "Collard Greens in a Hurry",
    "Lowcountry Oxtails With Ham Hocks",
    "ham hock gumbo",
    "Double Crust Peach Cobbler",
    "Garlic Cheese Grits",
    "Ham Hock and Lentil Soup",
    "Low(Er) Fat Collard Greens",
    "Okra Fritters",
    "Jalapeno Cheese Grits",
    "Creole Cabbage",
    "Collard Greens",
    "Ground Beef and Cabbage Stew",
    "Shrimp and Okra",
    "Bacon Cider Sauteed Collard Greens/Honey Mustard Drizzle",
    "Texas Country Cabbage",
    "Chili-Cheese Grits",
    "Ham and Okra",
    "Cajun Fish and Shrimp over Bacon Cheddar Cheese Grits",
    "Crock Pot Split Pea and Ham Hock Soup",
    "Mom\\'s Cabbage Soup",
    "Easy Collard Greens",
    "Green Beans and Ham Hocks",
    "Oven Fried Green Tomatoes and Okra",
    "Gorgonzola Cheese Grits",
    "Cheese Grits Souffle",
    "Mild Okra Fritters",
    "Fried Chicken Gizzards",
]
f'{expanded_query=}'

"expanded_query='what is a breakfast without eggs, and have milk,  apple,  pasta or  pizza, and contain calories from carbohydrates with desired range 0.0% to 45.0%?'"

In [13]:
#
# Query KBQA using the entities and expanded query
#
kbqa_query_data = {
    "intent": "constraint",
    "output": "N/A",
    "question": expanded_query,
    # we don't need to expand the query again
    "skip_expansion": True,
    "persona": persona,
    "entities": tags,
    "constrained_entities": constrained_entities,
    "similar_recipes": similar_recipes
}
kbqa_resp = requests.post(f"{KBQA_URL}/kbqa", json=kbqa_query_data)
kbqa_resp.json()['answer_list']

['Clean Whole Wheat Crepes',
 'Peanut Butter Cutlets',
 "Gb\\'s Breakfast Casserole",
 'Bacon and Egg Casserole',
 'Quiche']